In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# 텍스트 분류 실습 : 20 뉴스그룹 분류

### 텍스트 분류

: 특정 문서의 분류를 학습 데이터를 통해 학습 모델을 생성한 뒤 이 모델을 통해 다른 문서의 분류를 예측하는 것

**실습 예제**
- 사이킷런 내부에 가지고 있는 20 뉴스그룹 예제데이터
    - fetch_20newsgroups() API 이용
    - 뉴스그룹 분류를 위한 데이터
    

**실습 예제 분석 방법**
- 텍스트의 피처 벡터화 후 생성된 희소행렬을 이용해 분류 알고리즘 적용
    - 분류 알고리즘 : 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등
    

- 카운트 기반과 TF-IDF 기반의 벡터화를 차례로 적용하여 예측 성능 비교


- 피처 벡터화를 위한 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝


- Pipeline 객체를 통한 피처 벡터화 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝을 한번에 수행

## 1. 텍스트 정규화

#### fetch_20newsgroups()
- 인터넷에서 로컬 컴퓨터로 데이터를 먼저 내려받은 후 메모리로 데이터를 로딩
- 내컴퓨터의 [C:\Users\사용자폴더명\scikit_learn_data]에 저장

In [2]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [3]:
news_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
news_data.filenames

array(['C:\\Users\\gillhk\\scikit_learn_data\\20news_home\\20news-bydate-train\\rec.motorcycles\\104321',
       'C:\\Users\\gillhk\\scikit_learn_data\\20news_home\\20news-bydate-train\\rec.motorcycles\\103229',
       'C:\\Users\\gillhk\\scikit_learn_data\\20news_home\\20news-bydate-test\\sci.electronics\\54286',
       ...,
       'C:\\Users\\gillhk\\scikit_learn_data\\20news_home\\20news-bydate-train\\rec.autos\\102799',
       'C:\\Users\\gillhk\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.ibm.pc.hardware\\60175',
       'C:\\Users\\gillhk\\scikit_learn_data\\20news_home\\20news-bydate-train\\rec.sport.baseball\\104387'],
      dtype='<U96')

- load_xxx() API와 유사한 Key 값을 가짐
    - filenames : 로컬 컴퓨터에 저장하는 디렉터리와 파일명
    - target_names : target 클래스들의 이름

In [5]:
news_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
news_data.target

array([ 8,  8, 12, ...,  7,  3,  9])

In [7]:
pd.Series(news_data.target)

0         8
1         8
2        12
3        10
4         6
         ..
18841    19
18842     3
18843     7
18844     3
18845     9
Length: 18846, dtype: int32

#### 개별 데이터 추출

In [9]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 뉴스그룹 데이터 구성
    - 제목
    - 작성자
    - 소속
    - 이메일
    - 뉴스그룹 내용

#### 기사 내용만 가지고 텍스트 분석 진행
- 제목, 소속, 이메일 등의 헤더(header)와 푸터(footer) 정보들은 뉴스 그룹 분류에 높은 예측 성능을 가질 수 있으므로 제거함
    - remove 파라미터 사용

In [10]:
train_news = fetch_20newsgroups(subset='train', remove=('header','footer','quotes'), random_state= 156)

x_train = train_news.data
y_train = train_news.target
print(f'train dataset: {len(x_train)}')

train dataset: 11314


In [11]:
test_news = fetch_20newsgroups(subset='test', remove=('header','footer','quotes'), random_state= 156)

x_test = test_news.data
y_test = test_news.target
print(f'test dataset: {len(x_test)}')

test dataset: 7532


In [12]:
print(x_train[1])

From: cpage@two-step.seas.upenn.edu (Carter C. Page)
Subject: Re: Prayer in Jesus' Name
Organization: University of Pennsylvania
Lines: 46


	"And in that day you will ask Me no question.  Truly, truly, I say to 
	you, if you shall ask the Father for anything, He will give it to you 
	in my name.  Until now you have asked for nothing in My name; ask, and 
	you will receive, that your joy may be made full."
				-John 16:23-24

I don't believe that we necessarily have to say " . . . In Christ's name.  
Amen," for our prayers to be heard, but it glorifies the Son, when we 
acknowledge that our prayer is made possible by Him.  I believe that just as 
those who were saved in the OT, could only be saved because Jesus would one day
reconcile God to man, He is the only reason their prayers would be heard by 
God.

	For all of us have become like one who is unclean,
	And all our righteous deeds are like a filthy garment;
	And all of us wither like a leaf,
	and our iniquities, like the wind, tak

## 2. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

### 2.1 CountVectorizer를 이용해 텍스트데이터 피처 벡터화


**테스트 데이터에서 CountVectorizer를 적용할 때 주의점**: 
- 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환(transform)해야 함
    - 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아짐
- 테스트 데이터 피처 벡터화할 때 fit_transform()을 사용하면 안됨
    - 테스트 데이터를 fit_transform()을 이용해 벡터화하면 피처개수가 학습데이터와 달라지므로

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
cnt = CountVectorizer()

# 학습
cnt.fit(x_train)

# 학습 데이터 피처 벡터화
x_train_cnt = cnt.transform(x_train)

# 테스트 데이터 피처 벡테화
x_test_cnt = cnt.transform(x_test)

print(f'{x_train_cnt.shape}, {x_test_cnt.shape}')

(11314, 120756), (7532, 120756)


### 2.2 로지스틱 회귀를 적용해 뉴스그룹 분류 예측

**Count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀 모델 적용하여 예측**

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(random_state= 0)
lr.fit(x_train_cnt, y_train)
pred = lr.predict(x_test_cnt)
print(f'정확도: {accuracy_score(y_test, pred):.4f}')

정확도: 0.7569


### 2.3 TF-IDF를 이용한 피처벡터화 및 로지스틱 회귀로 예측

**TF-IDF 기반으로 벡터화를 변경한 데이터 세트에 대해 로지스틱 회귀 적용하여 예측**

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tf = TfidfVectorizer()

# 학습
tf.fit(x_train)

# 학습 데이터 피처 벡터화
x_train_tf = tf.transform(x_train)

# 테스트 데이터 피처 벡테화
x_test_tf = tf.transform(x_test)

print(f'{x_train_tf.shape}, {x_test_tf.shape}')

(11314, 120756), (7532, 120756)


In [21]:
lr_tf = LogisticRegression(random_state= 0)
lr_tf.fit(x_train_tf, y_train)
pred_tf = lr_tf.predict(x_test_tf)
print(f'정확도: {accuracy_score(y_test, pred_tf):.4f}')

정확도: 0.7841


**텍스트 분석에서 머신러닝 모델 성능을 향상시키는 주요 방법 2가지**
- 최적의 ML 알고리즘 선택하는 것
- 최상의 피처 전처리를 수행하는 것


=> 텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반 머신러닝 성능에  큰 영향을 미칠 수 있음

**TF-IDF 벡터화에서 좀 더 다양한 파라미터를 적용하여 예측 수행**

: TfidfVectorizer 클래스의 파라미터 변경

- stop_words : None에서 'english'로 변경
- ngram_range : (1,1)에서 (1,2)로 변경
- max_df=300으로 변경 (전체 문서에서 300개 이하로 나타나는 단어만 피처로 추출)

In [22]:
# ngram -> 연속된 단어로 추출 ex)  "This is a pen" -> 2-그램을 추출하면 "This is", "is a", "a pen"
tf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df= 300)

tf.fit(x_train)
x_train_tf = tf.transform(x_train)
x_test_tf = tf.transform(x_test)

lr_tf = LogisticRegression(random_state= 0)
lr_tf.fit(x_train_tf, y_train)
pred_tf = lr_tf.predict(x_test_tf)
print(f'정확도: {accuracy_score(y_test, pred_tf):.4f}')

정확도: 0.7744


### 2.3 GridSearchCV를 이용해 로지스틱 회귀의 하이퍼파라미터 최적화 수행

In [23]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'C': [0.01, 0.1, 1,5,10]}
grid = GridSearchCV(estimator= lr, param_grid= params, cv=3, scoring='accuracy', verbose= 1)

grid.fit(x_train_tf, y_train)

print(f'최적의 하이퍼 파라미터: {grid.best_params_}')
print(f'최적의 성능: {grid.best_score_}')

Fitting 3 folds for each of 5 candidates, totalling 15 fits


In [ ]:
# 최적의 C값으로 학습된 grid로 예측
params = {'C': [0.01, 0.1, 1,5,10]}
grid_best = GridSearchCV(estimator= lr, param_grid= params, cv=3, scoring='accuracy', verbose= 1)
pred = grid_best.predict(x_test_tf)
print(f'최적의 모델 예측성능: {accuracy_score(y_test, pred):.4f}')

## 3. 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합

**사이킷런의 Pipeline 클래스를 사용하여 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행**

**Pipeline이란?**
- 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 '수도관(pipe)에서 물이 흐르듯' 한꺼번에 스트림 기반으로 처리한다는 의미

**파이프라인 이용의 장점**
- 데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있음
- 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있어 수행시간을 절약할 수 있음

**사이킷런의 파이프라인은 모든 데이터 전처리 작업과 Estimator를 결합할 수 있음**
- 예. 스케일링 또는 벡터 정규화 + PCA 등의 변환작업 +  분류, 회귀 등의 Estimator를 한 번에 결합

### 3.1 텍스트 분류 예제 코드를 Pipeline을 이용하여 다시 작성

**Pipeline 객체를 사용하여 TfidfVectorizer와 LogisticRegressor를 위한 파라미터 지정**

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([('TF-IDF', TfidfVectorizer(stop_words='english'), ngram_range=(1,2), max_df= 300),
('LR', LogisticRegression(c=10, max_iter= 150))])

In [ ]:
pipe.fit(x_train, y_train)
pred = pipe.predict(x_test)
print(f'Pipeline을 이용한 예측성능: {accuracy_score(y_test, pred):.4f}')

### 3.2 GridSearchCV에 Pipeline을 입력하여 파라미터 최적화

In [ ]:
pipe = Pipeline([('TF-IDF', TfidfVectorizer(stop_words='english')), ('LR', LogisticRegression())])

params = {'TF-IDF__ngram_range':[(1,1),(1,2)],
         'TF-IDF__max_df':[100,300],
         'LR__C': [5,10]}

grid_pipe = GridSearchCV(pipe, param_grid= params, cv=3, scoring='accuracy', verbose=1)
grid_pipe.fit(x_train, y_train)
print(f'최적의 하이퍼 파라미터: {grid_pipe.best_params_}')
print(f'최적의 성능: {grid_pipe.best_score_}')

pred = grid_pipe.predict(x_test)
print(f'Pipeline 이용한 최적모델 예측 정확도: {accuracy_score(y_test, pred):.4f}')

----